In [1]:
import os
os. chdir('../')

## CREATING KENLM MODEL BASED ON DATASET

In [2]:
import torch
import pandas as pd
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from flash.audio import SpeechRecognition, SpeechRecognitionData
from training.wav2vec2_finetune import Wav2Vec2SpeechRecognition, SpeechRecognitionData
from constants.mir_constants import WAV2VEC2_ARGS
from jiwer import wer
from datasets import load_dataset,Dataset, Audio
import json
from dataclasses import dataclass,asdict
from pyctcdecode import build_ctcdecoder
from transformers import AutoTokenizer
import multiprocessing

/home/users/gmenon/.local/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/users/gmenon/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
BASE_PATH = '/scratch/users/gmenon/'
song_metadata_en_demucs_cleaned = pd.read_csv("/scratch/users/gmenon/song_metadata_en_demucs_cleaned.csv")
kenlm_text_lst = list(song_metadata_en_demucs_cleaned['transcription_capitalized'] + str("."))
with open(BASE_PATH + 'lyrics.txt','w') as tfile:
    tfile.write('\n'.join(kenlm_text_lst))

In [ ]:
os.system(f"/scratch/users/gmenon/kenlm/build/bin/lmplz -o 5 --text /scratch/users/gmenon/lyrics.txt --arpa /scratch/users/gmenon/lyrics.arpa")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
vocab_dict = tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
decoder = build_ctcdecoder(list(sorted_dict.keys()),'/scratch/users/gmenon/lyrics.arpa')
decoder.save_to_dir(BASE_PATH + "wav2vec2-with_lm/")

## LOADING SPEECH EVALUATION MODEL, PROCESSOR AND DATASET

In [3]:
WAV2VEC2_ARGS.MODEL_SAVE_PATH = "/scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adam.pt"
print(json.dumps(asdict(WAV2VEC2_ARGS), indent = 4))

{
    "TRAIN_FILE_PATH": "/scratch/users/gmenon/train_song_metadata_en_demucs_cleaned_filtered_095.csv",
    "TEST_FILE_PATH": "/scratch/users/gmenon/validation_song_metadata_en_demucs_cleaned_filtered_005.csv",
    "MODEL_BACKBONE": "facebook/wav2vec2-large-960h-lv60-self",
    "BATCH_SIZE": 1,
    "NUM_EPOCHS": 15,
    "MODEL_SAVE_PATH": "/scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adam.pt",
    "FINETUNE_STRATEGY": [
        "freeze_unfreeze",
        10
    ],
    "LR_SCHEDULER": "reduce_on_plateau_schedule"
}


In [4]:
speech_recognition_task = Wav2Vec2SpeechRecognition(wav2vec2_args=WAV2VEC2_ARGS)
processor = Wav2Vec2Processor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)

wandb: Currently logged in as: greeshmasmenon (songslyricstranscription). Use `wandb login --relogin` to force relogin


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
test_data = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
audio_dataset = Dataset.from_dict({"audio": list(test_data["consolidated_file_path"]), "transcription": list(test_data["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16000))
audio_dataset[0]["audio"]["array"]

array([ 0.00768129,  0.00689538, -0.0072176 , ...,  0.00820587,
       -0.03958851,  0.        ])

In [8]:

def beam_search_best_parameter_search(alpha_value,beta_value):
    "For a particular alpha and beta value, it returns the greedy and beam search error"
    labels = list(processor.tokenizer.get_vocab().keys())
    print(f"Model Save Path Location = {WAV2VEC2_ARGS.MODEL_SAVE_PATH}")
    model = SpeechRecognition.load_from_checkpoint(WAV2VEC2_ARGS.MODEL_SAVE_PATH)
    greedy_predictions = []
    beamsearch_predictions = []
    transcripts = []
    greedy_predictions_transformed=[]
    decoder = build_ctcdecoder(
        labels,
        kenlm_model_path="/scratch/users/gmenon/lyrics.arpa",
        alpha=alpha_value,
        beta=beta_value)
    
    for i in range(len(audio_dataset)):
        y_hat = model.model(torch.Tensor(processor(audio_dataset[i]["audio"]["array"],sampling_rate = 16_000)["input_values"][0]).unsqueeze(0))
        transcript = audio_dataset["transcription"][i]
        predicted_ids = torch.argmax(y_hat.logits, dim=-1)
        pred_greedy = processor.batch_decode(predicted_ids)
        pred_beam = decoder.decode(logits=y_hat.logits.detach().numpy().squeeze(),beam_width=100)
        transcripts.append(transcript)
        greedy_predictions.append(pred_greedy)
        beamsearch_predictions.append(pred_beam)
    
    for predictions in greedy_predictions:
        greedy_predictions_transformed.append(predictions[0])
    
    hypothesis = beamsearch_predictions
    reference = transcripts
    beam_error = wer(reference, hypothesis)
    
    hypothesis = greedy_predictions_transformed
    reference = transcripts
    greedy_error = wer(reference, hypothesis)
    return greedy_error,beam_error


In [9]:
from itertools import product
alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
betas = [3.0,2.0,1.0,0.9,0.8,0.7,0.6,0.5]

for alpha,beta in product(alphas,betas):
    greedy_error,beam_error = beam_search_best_parameter_search(alpha,beta)
    print(f"alpha = {alpha}, beta = {beta}, greedy_wer = {greedy_error}, beam_wer = {beam_error}")
    

Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adam.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 0.1, beta = 3.0, greedy_wer = 0.7413305024769993, beam_wer = 0.8535031847133758
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adam.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 0.1, beta = 2.0, greedy_wer = 0.7413305024769993, beam_wer = 0.7887473460721869
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adam.pt


...

alpha = 1.0, beta = 0.7, greedy_wer = 0.7413305024769993, beam_wer = 0.613588110403397
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adam.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 1.0, beta = 0.6, greedy_wer = 0.7413305024769993, beam_wer = 0.6139419674451522
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adam.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 1.0, beta = 0.5, greedy_wer = 0.7413305024769993, beam_wer = 0.6139419674451522


In [10]:
WAV2VEC2_ARGS.MODEL_SAVE_PATH = "/scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_finetuned_15epochs_freeze_deepspeed.pt"


In [11]:

def beam_search_best_parameter_search(alpha_value,beta_value):
    "For a particular alpha and beta value, it returns the greedy and beam search error"
    labels = list(processor.tokenizer.get_vocab().keys())
    print(f"Model Save Path Location = {WAV2VEC2_ARGS.MODEL_SAVE_PATH}")
    model = SpeechRecognition.load_from_checkpoint(WAV2VEC2_ARGS.MODEL_SAVE_PATH)
    greedy_predictions = []
    beamsearch_predictions = []
    transcripts = []
    greedy_predictions_transformed=[]
    decoder = build_ctcdecoder(
        labels,
        kenlm_model_path="/scratch/users/gmenon/lyrics.arpa",
        alpha=alpha_value,
        beta=beta_value)
    
    for i in range(len(audio_dataset)):
        y_hat = model.model(torch.Tensor(processor(audio_dataset[i]["audio"]["array"],sampling_rate = 16_000)["input_values"][0]).unsqueeze(0))
        transcript = audio_dataset["transcription"][i]
        predicted_ids = torch.argmax(y_hat.logits, dim=-1)
        pred_greedy = processor.batch_decode(predicted_ids)
        pred_beam = decoder.decode(logits=y_hat.logits.detach().numpy().squeeze(),beam_width=100)
        transcripts.append(transcript)
        greedy_predictions.append(pred_greedy)
        beamsearch_predictions.append(pred_beam)
    
    for predictions in greedy_predictions:
        greedy_predictions_transformed.append(predictions[0])
    
    hypothesis = beamsearch_predictions
    reference = transcripts
    beam_error = wer(reference, hypothesis)
    
    hypothesis = greedy_predictions_transformed
    reference = transcripts
    greedy_error = wer(reference, hypothesis)
    return greedy_error,beam_error


In [ ]:
from itertools import product
alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
betas = [3.0,2.0,1.0,0.9,0.8,0.7,0.6,0.5]

for alpha,beta in product(alphas,betas):
    greedy_error,beam_error = beam_search_best_parameter_search(alpha,beta)
    print(f"alpha = {alpha}, beta = {beta}, greedy_wer = {greedy_error}, beam_wer = {beam_error}")
    

Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_finetuned_15epochs_freeze_deepspeed.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 0.1, beta = 3.0, greedy_wer = 0.9171974522292994, beam_wer = 1.0314932767162066
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_finetuned_15epochs_freeze_deepspeed.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 0.1, beta = 2.0, greedy_wer = 0.9171974522292994, beam_wer = 0.93241330502477
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_finetuned_15epochs_freeze_deepspeed.pt


...

Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 0.4, beta = 0.8, greedy_wer = 0.9171974522292994, beam_wer = 0.7947629157820241
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_finetuned_15epochs_freeze_deepspeed.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


alpha = 0.4, beta = 0.7, greedy_wer = 0.9171974522292994, beam_wer = 0.7997169143665959
Model Save Path Location = /scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_finetuned_15epochs_freeze_deepspeed.pt


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading the LM will be faster if you build a binary file.
Reading /scratch/users/gmenon/lyrics.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.
